# Inicializace

In [4]:
from collections import deque               # Ukladani stavu
from __future__ import division             # Deleni realnych cisel (kvuli verzi Pythonu 2.6)
import sys                                  # Pro navazani knihovny Open AI Gym
import numpy as np                          # Knihovna pro operace s maticemi
sys.path.append('/home/xbucha02/libraries') # Adresa knihovny Open AI Gym
import gym                                  # Knihovna Open AI Gym
#from gym import wrappers                    # Pomucka na nahravani
env = gym.make('CartPole-v0')            # Konkretni hra z Open AI Gym
#env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True) #Nahravani zaznamu
#gpuMemoryUsage=1                            # Vyuziti pameti GPU
#import tensorflow as tf                     # Knihovna TensorFlow pro sdileni GPU
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
#set_session(tf.Session(config=config))
import random

# Parametry
observetime = 500                          # Pocet framu kazde hry
episodes = 1000                             # Pocet epizod
startEpsilon = 1.0                         # Pravdepodobnost konani nahodneho tahu na zacatku
endEpsilon = 0.005                           # Pravdepodobnost konani nahodneho tahu na konci
currentEpsilon = startEpsilon              # Soucasna pravdepodobnost konani nahodneho tahu
epsilonDiminution = (startEpsilon - endEpsilon)/(episodes * 5)                 # Hodnota snizovani epsilonu
gamma = 0.99                               # Discount faktor
minibatchSize = 32                         # Velikost minibatche
actionCount = env.action_space.n           # Pocet vstupu do prostredi
stateSize = env.observation_space.shape[0] # Pocet vystupu z prostredi
learningRate = 0.001                       # Learning rate
memorySize = 2000                         # Velikost Replay memory

# Model NN

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import backend as K

def _huber_loss(target, prediction):
    # sqrt(1+error^2)-1
    error = prediction - target
    return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)

def getDQN(actionCount,stateSize):
    net = Sequential()
    net.add(Dense(32, activation="relu", input_dim=stateSize))
    net.add(Dense(16, activation="relu"))
    net.add(Dense(actionCount, activation="linear"))
    
    net.summary()
    
    net.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(lr=learningRate), metrics=['accuracy'])
    
    return net

def loadDQN(net, name):
    net.load_weights(name)

def saveDQN(net, name):
    net.save_weights(name)

# Algoritmus DQN

In [6]:
net = getDQN(actionCount, stateSize)
netTarget = getDQN(actionCount, stateSize)
observation = env.reset()                    # Reset prostredi
memory = deque(maxlen=memorySize)

global scores
scores = []
global episodes
episodesList = []

for eps in range (episodes):
    
    #D = deque() #vyprazdneni D
    state = env.reset()
    state = np.reshape(state, [1, stateSize])
    #env.render()

    #q = 0
    for t in range(observetime):
        #if q == 0:
            
        #else:
        #    if q == 4:
        #        q = 0
        #    else:
        #        q = q + 1
        if currentEpsilon > endEpsilon:
            currentEpsilon = currentEpsilon - epsilonDiminution
            
        if np.random.rand() <= currentEpsilon:
            action = np.random.randint(0, actionCount, size=1)[0]
        else:
            Q = net.predict(state)

            action = np.argmax(Q)

        observation_new, reward, done, info = env.step(int(action))
        
        #reward = reward if not done else -10
        
        #if done and t != 199:
        #    reward = 10000
        
        next_state = np.reshape(observation_new, [1, stateSize])

        memory.append((state, action, reward, next_state, done))

        if len(memory) >= minibatchSize:
            minibatch = random.sample(memory, minibatchSize) #z D vybere pocet mb_size samplu

            for i in range(0, minibatchSize):

                stateMb = minibatch[i][0]
                actionMb = minibatch[i][1]
                rewardMb = minibatch[i][2]
                state_newMb = minibatch[i][3]
                doneMb = minibatch[i][4]

                target_f = net.predict(stateMb)

                if doneMb:
                    target_f[0][actionMb] = rewardMb
                else:
                    aNet = net.predict(state_newMb)[0]
                    tNet = netTarget.predict(state_newMb)[0]
                    target_f[0][actionMb] = rewardMb + gamma * tNet[np.argmax(aNet)]

                net.fit(stateMb, target_f, epochs=1, verbose=0)
        
        state = next_state         # Update state
        
        if eps % 25 == 0:
            saveDQN(net, "./DQN-CartPole-v0.h5")
        
        if done:
            print("Episode: {}/{}, epsilon: {:.2}, score: {}".format(eps, episodes, currentEpsilon, t))
            scores.append(t)
            episodesList.append(eps)
            netTarget.set_weights(net.get_weights())
            break

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                160       
_________________________________________________________________
dense_8 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 34        
Total params: 722
Trainable params: 722
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_11 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_12 (De

Episode: 152/1000, epsilon: 0.0048, score: 108
Episode: 153/1000, epsilon: 0.0048, score: 93
Episode: 154/1000, epsilon: 0.0048, score: 99
Episode: 155/1000, epsilon: 0.0048, score: 98
Episode: 156/1000, epsilon: 0.0048, score: 96
Episode: 157/1000, epsilon: 0.0048, score: 92
Episode: 158/1000, epsilon: 0.0048, score: 92
Episode: 159/1000, epsilon: 0.0048, score: 103
Episode: 160/1000, epsilon: 0.0048, score: 104
Episode: 161/1000, epsilon: 0.0048, score: 124
Episode: 162/1000, epsilon: 0.0048, score: 145
Episode: 163/1000, epsilon: 0.0048, score: 145
Episode: 164/1000, epsilon: 0.0048, score: 142
Episode: 165/1000, epsilon: 0.0048, score: 139
Episode: 166/1000, epsilon: 0.0048, score: 124
Episode: 167/1000, epsilon: 0.0048, score: 134
Episode: 168/1000, epsilon: 0.0048, score: 154
Episode: 169/1000, epsilon: 0.0048, score: 109
Episode: 170/1000, epsilon: 0.0048, score: 123
Episode: 171/1000, epsilon: 0.0048, score: 143
Episode: 172/1000, epsilon: 0.0048, score: 153
Episode: 173/1000, 

Episode: 327/1000, epsilon: 0.0048, score: 78
Episode: 328/1000, epsilon: 0.0048, score: 86
Episode: 329/1000, epsilon: 0.0048, score: 94
Episode: 330/1000, epsilon: 0.0048, score: 92
Episode: 331/1000, epsilon: 0.0048, score: 100
Episode: 332/1000, epsilon: 0.0048, score: 126
Episode: 333/1000, epsilon: 0.0048, score: 112
Episode: 334/1000, epsilon: 0.0048, score: 27
Episode: 335/1000, epsilon: 0.0048, score: 19
Episode: 336/1000, epsilon: 0.0048, score: 45
Episode: 337/1000, epsilon: 0.0048, score: 199
Episode: 338/1000, epsilon: 0.0048, score: 135
Episode: 339/1000, epsilon: 0.0048, score: 126
Episode: 340/1000, epsilon: 0.0048, score: 135
Episode: 341/1000, epsilon: 0.0048, score: 152
Episode: 342/1000, epsilon: 0.0048, score: 131
Episode: 343/1000, epsilon: 0.0048, score: 199
Episode: 344/1000, epsilon: 0.0048, score: 199
Episode: 345/1000, epsilon: 0.0048, score: 199
Episode: 346/1000, epsilon: 0.0048, score: 199
Episode: 347/1000, epsilon: 0.0048, score: 199
Episode: 348/1000, e

Episode: 502/1000, epsilon: 0.0048, score: 138
Episode: 503/1000, epsilon: 0.0048, score: 162
Episode: 504/1000, epsilon: 0.0048, score: 134
Episode: 505/1000, epsilon: 0.0048, score: 141
Episode: 506/1000, epsilon: 0.0048, score: 189
Episode: 507/1000, epsilon: 0.0048, score: 199
Episode: 508/1000, epsilon: 0.0048, score: 199
Episode: 509/1000, epsilon: 0.0048, score: 199
Episode: 510/1000, epsilon: 0.0048, score: 159
Episode: 511/1000, epsilon: 0.0048, score: 176
Episode: 512/1000, epsilon: 0.0048, score: 145
Episode: 513/1000, epsilon: 0.0048, score: 119
Episode: 514/1000, epsilon: 0.0048, score: 150
Episode: 515/1000, epsilon: 0.0048, score: 134
Episode: 516/1000, epsilon: 0.0048, score: 140
Episode: 517/1000, epsilon: 0.0048, score: 138
Episode: 518/1000, epsilon: 0.0048, score: 142
Episode: 519/1000, epsilon: 0.0048, score: 138
Episode: 520/1000, epsilon: 0.0048, score: 139
Episode: 521/1000, epsilon: 0.0048, score: 132
Episode: 522/1000, epsilon: 0.0048, score: 144
Episode: 523/

KeyboardInterrupt: 

# Výsledky

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt
import copy

score1 = copy.copy(scores)
score2 = copy.copy(scores)
score3 = copy.copy(scores)

for i in range (len(scores)):
    if i > 1 and i < (len(scores)-2):
        score1[i] = (scores[i - 2] + scores[i - 1] + scores[i] + scores[i + 1] + scores[i + 2])/5
        
for i in range (len(scores)):
    if i > 4 and i < (len(scores)-5):
        score2[i] += scores[i - 5] + scores[i - 4] + scores[i - 3] + scores[i - 2] + scores[i - 1]
        score2[i] += scores[i + 5] + scores[i + 4] + scores[i + 3] + scores[i + 2] + scores[i + 1]
        score2[i] = score2[i]/11

for i in range (len(scores)):
    if i > 49 and i < (len(scores) - 50):
        for e in range (1,50):
            score3[i] += scores[i - e] + scores[i + e] 
        score3[i] = score3[i]/101      

plt.plot(episodesList, scores, 'ro')
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Vysledky")
plt.show()

plt.plot(scores)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace vysledku")
plt.show()

plt.plot(score1)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace vysledku (filtr - prumer 5-ti prvku)")
plt.show()

plt.plot(score2)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace vysledku  (filtr - prumer 11-cti prvku)")
plt.show()

plt.plot(score3)
plt.ylabel("Score")
plt.xlabel("Episodes")
plt.title("Funkce interpolace vysledku  (filtr - prumer 101 prvku)")
plt.show()

saveDQN(net, "./DQN-CartPole-v0.h5")

# Hra

In [ ]:
netForPlay = getDQN(actionCount, stateSize)
loadDQN(netForPlay, "./DQN-CartPole-v0.h5")
observation = env.reset()

for i in range (100):
    state = env.reset()
    state = np.reshape(state, [1, stateSize])
    
    total_reward = 0
    done = False
    
    while not done:
        #env.render()
        Q = netForPlay.predict(state)   
        action = np.argmax(Q)  
        observation_new, reward, done, info = env.step(int(action))

        next_state = np.reshape(observation_new, [1, stateSize])
        state = next_state

        total_reward += reward

    print('{}. Game - score: {}'.format(i, total_reward))